---
### Подготовка

Импорт библиотек

In [31]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from IPython.display import display
plt.rc('font', family='Verdana')
from sklearn.decomposition import PCA, KernelPCA
import mlxtend as ml
from mlxtend.frequent_patterns import apriori, fpgrowth
import itertools
from typing import Dict, List
from mlxtend.preprocessing import TransactionEncoder
encoder = TransactionEncoder()
import collections

<br />
Подготовка данных из задания

Дан набор данных:

| tid              | itemset |
| ---------------- | ------- |
| *t<sub>1</sub>*  | *DFG*    |
| *t<sub>2</sub>*  | *AG*    |
| *t<sub>3</sub>*  | *BF*    |
| *t<sub>4</sub>*  | *CEG*    |
| *t<sub>5</sub>*  | *BDF*    |
| *t<sub>6</sub>*  | *CF*    |
| *t<sub>7</sub>*  | *CEG*    |
| *t<sub>8</sub>*  | *ABCEG*    |
| *t<sub>9</sub>*  | *ABF*   |
| *t<sub>10</sub>* | *BEG*   |

In [32]:
tid = ['t1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10']
itemset = ['DFG', 'AG', 'BF', 'CEG', 'BDF', 'CF', 'CEG', 'ABCEG', 'ABF', 'BEG']
min_support = 0.3

### Задание №1 
### Алгоритм Apriori

**1.1**. Минимальный уровень поддержки равен **3 / 10** 

*Самостоятельно* реализуйте алгоритм **Apriori** и продемонстрируйте, как алгоритм перебирает предложенный набор данных. Выведите результат работы алгоритма.<br /><br />

In [33]:
def apriori(df: pd.DataFrame, min_support: float) -> Dict[int, List[List[str]]]:
    transaction_matrix = df.values
    transaction_count = transaction_matrix.shape[0]
    support_counts = np.array(np.sum(transaction_matrix, axis=0) / transaction_count)
    item_id_to_name = {i: item for i, item in enumerate(df.columns)}
    item_ids = np.arange(transaction_matrix.shape[1])
    item_set_supports = {1: support_counts[support_counts >= min_support]}
    frequent_item_sets = {1: item_ids[support_counts >= min_support].reshape(-1, 1)}

    k = 2
    while True:
        new_item_sets = []
        leaves = set(frequent_item_sets[k-1].flatten())
        for item_set in frequent_item_sets[k-1]:
            max_item_id = max(item_set)
            new_leaves = [x for x in leaves if x > max_item_id]
            last_tuple = tuple(item_set)
            for new_leaf in new_leaves:
                new_item_sets.append(last_tuple + (new_leaf,))
        
        new_item_sets = np.array(new_item_sets)
        if (new_item_sets.size == 0):
            break
        
        bool_mask = np.array(np.all(transaction_matrix[:, new_item_sets], axis=2))
        support_counts = np.array(np.sum(bool_mask, axis=0) / transaction_count)
        mask = (support_counts >= min_support)
        if mask.any():
            item_set_supports[k] = np.array(support_counts[mask])
            frequent_item_sets[k] = np.array(new_item_sets[mask])
        else:
            break
        
        k += 1
        
    result = {}
    for k in frequent_item_sets:
        for item_set, support in zip(frequent_item_sets[k], item_set_supports[k]):
            result[tuple(item_id_to_name[i] for i in item_set)] = support
    return result

encoder.fit(itemset)
encodedData = encoder.transform(itemset)
data = pd.DataFrame(encodedData, columns = encoder.columns_, index = tid)
apriori(data, min_support)

{('A',): 0.3,
 ('B',): 0.5,
 ('C',): 0.4,
 ('E',): 0.4,
 ('F',): 0.5,
 ('G',): 0.6,
 ('B', 'F'): 0.3,
 ('C', 'E'): 0.3,
 ('C', 'G'): 0.3,
 ('E', 'G'): 0.4,
 ('C', 'E', 'G'): 0.3}

**1.2**. Воспользуйтесь алгоритмом **Apriori** из библиотеки **MLxtend** и выведите результат его работы. Сравните результаты собственной и библиотечной реализации.<br />

In [34]:
dataset = [['D', 'F', 'G'],
           ['A', 'G'],
           ['B', 'F'],
           ['C', 'E', 'G'],
           ['B', 'D','F'],
           ['C', 'F'],
           ['C', 'E', 'G'],
           ['A', 'B', 'C', 'E', 'G'],
           ['A', 'B', 'F'],
           ['B', 'E', 'G']]

te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)

ml.frequent_patterns.apriori(df, min_support=0.3, use_colnames=True)

,support,itemsets
0,0.3,(A)
1,0.5,(B)
2,0.4,(C)
3,0.4,(E)
4,0.5,(F)
5,0.6,(G)
6,0.3,"(F, B)"
7,0.3,"(E, C)"
8,0.3,"(G, C)"
9,0.4,"(G, E)"


### Алгоритм FPGrowth

**1.3**. Минимальный уровень поддержки равен **5 / 10**
*Самостоятельно* реализуйте алгоритм **FPGrowth** и продемонстрируйте, как алгоритм перебирает предложенный набор данных. Выведите результат работы алгоритма.<br />

In [35]:
TE = TransactionEncoder()
teArray = TE.fit(itemset).transform(itemset)
dataTask1 = pd.DataFrame(teArray, columns=TE.columns_, index=tid)

class FPNode(object):
    def __init__(self, item, cnt, parent):
        self.item = item
        self.cnt = cnt
        self.parent = parent
        self.children = collections.defaultdict(FPNode)

        if parent is not None:
            parent.children[item] = self

    def pathFromRoot(self):
        path = []
        if self.item is None:
            return path

        node = self.parent
        while node.item is not None:
            path.append(node.item)
            node = node.parent

        path.reverse()
        return path


class FPTree(object):
    def __init__(self, rank):
        self.root = FPNode(None, 0, None)
        self.nodes = collections.defaultdict(list)
        self.projItems = []
        self.rank = rank

    def p_tree(self, p_item, minimal):
        branches = []
        cnt = collections.defaultdict(int)
        for node in self.nodes[p_item]:
            branch = node.pathFromRoot()
            branches.append(branch)
            for item in branch:
                cnt[item] += node.cnt

        items = [item for item in cnt if cnt[item] >= minimal]
        items.sort(key=cnt.get)
        rank = {item: i for i, item in enumerate(items)}

        p_tree_ret = self.sorting(branches, p_item, rank)

        return p_tree_ret

    def sorting(self, branches, p_item, rank):
        p_tree = FPTree(rank)
        for idx, branch in enumerate(branches):
            branch = sorted([i for i in branch if i in rank], key=rank.get, reverse=True)
            p_tree.itemset(branch, self.nodes[p_item][idx].cnt)
        p_tree.projItems = self.projItems + [p_item]
        return p_tree

    def itemset(self, itemset, cnt):
        self.root.cnt += cnt

        if len(itemset) == 0:
            return

        index = 0
        node = self.root
        self.itemset_loop(cnt, index, itemset, node)

    def itemset_loop(self, cnt, index, itemset, node):
        for item in itemset:
            if item in node.children:
                child = node.children[item]
                child.cnt += cnt
                node = child
                index += 1
            else:
                break
        for item in itemset[index:]:
            nodeChild = FPNode(item, cnt, node)
            self.nodes[item].append(nodeChild)
            node = nodeChild

    def isPath(self):
        if len(self.root.children) > 1:
            return False
        for i in self.nodes:
            if len(self.nodes[i]) > 1 or len(self.nodes[i][0].children) > 1:
                return False
        return True

    def get_one(self, cnt, mapping, output):
        p_items = [str(mapping[i]) for i in self.projItems]
        output = [str(mapping[i]) for i in output]
        p_items = ", ".join(p_items)
        if p_items:
            output = [", ".join([p_items, i]) for i in output]
        print("%d from [%s]:" % (cnt, p_items))
        for item in output:
            print("[%s]" % item)


def newTree(DF, minimal, d_size, mapping, printSteps):
    itemsets = DF.values

    supp = np.array(np.sum(itemsets, axis=0) / d_size)

    items = np.nonzero(supp >= minimal)[0]

    mask = supp[items].argsort()
    rank = {item: i for i, item in enumerate(items[mask])}

    if (printSteps):
        print([str(mapping[item]) for item in np.flip(items[mask])])

    tree = FPTree(rank)
    if (printSteps):
        print("\nSorted:")
    for i in range(d_size):
        notNull = np.where(itemsets[i, :])[0]
        itemset = [item for item in notNull if item in rank]
        itemset.sort(key=rank.get, reverse=True)
        tree.itemset(itemset, 1)
        if (printSteps):
            print([str(mapping[item]) for item in itemset])
    if (printSteps):
        print("\n")

    return tree


def FPGrowth(dataFrame, minsupp, printSteps=False):
    mapping = {idx: item for idx, item in enumerate(dataFrame.columns)}
    size = len(dataFrame.index);

    tree = newTree(dataFrame, minsupp, size, mapping, printSteps)
    minsupNum = np.ceil(minsupp * size)
    generator = one(tree, minsupNum, mapping, printSteps)

    itemsets = []
    supports = []
    for sup, iset in generator:
        itemsets.append(iset)
        supports.append(sup / size)

    resultFrame = pd.DataFrame({"supports": supports, "itemsets": itemsets})
    resultFrame["itemsets"] = resultFrame["itemsets"].apply(lambda x: [mapping[i] for i in x])

    return resultFrame


def one(tree, minimal, mapping, get_all):
    global outputlist
    cnt = 0
    items = tree.nodes.keys()

    if (get_all):
        outputlist = []

    if tree.isPath():
        cnt = yield from treePath(cnt, items, get_all, tree)
    else:
        cnt = yield from treeNotPath(cnt, items, get_all, outputlist, tree)

    if (get_all):
        tree.get_one(cnt, mapping, outputlist)

    if not tree.isPath():
        for item in items:
            projTree = tree.p_tree(item, minimal)
            for sup, iset in one(projTree, minimal, mapping, get_all):
                yield sup, iset


def treeNotPath(cnt, items, get_all, printlist, tree):
    for item in items:
        cnt += 1
        support = sum([node.cnt for node in tree.nodes[item]])
        if (get_all):
            printlist.append(item)
        yield support, tree.projItems + [item]
    return cnt


def treePath(cnt, items, get_all, tree):
    global outputlist
    newSize = len(items) + 1
    for i in range(1, newSize):
        for itemset in itertools.combinations(items, i):
            cnt += 1
            support = min([tree.nodes[i][0].cnt for i in itemset])
            if (get_all):
                outputlist += list(itemset)
            yield support, tree.projItems + list(itemset)
    return cnt

print(FPGrowth(dataTask1, 0.5, True))

['G', 'F', 'B']

Sorted:
['G', 'F']
['G']
['F', 'B']
['G']
['F', 'B']
['F']
['G']
['G', 'B']
['F', 'B']
['G', 'B']


3 from []:
[G]
[F]
[B]
0 from [G]:
0 from [F]:
0 from [B]:
   supports itemsets
0       0.6      [G]
1       0.5      [F]
2       0.5      [B]


**1.4**. Воспользуйтесь алгоритмом **FPGrowth** из библиотеки **MLxtend** и выведите результат его работы. Сравните результаты собственной и библиотечной реализации<br /><br />

In [36]:
display(fpgrowth(df, min_support = 0.5, use_colnames = True))

,support,itemsets
0,0.6,(G)
1,0.5,(F)
2,0.5,(B)


### Задание №2

Был получен следующий набор данных:

| tid  | itemset |
| ---- | ------- |
| *t<sub>1</sub>*  | *BCFGHILNQTUW[_h*    |
| *t<sub>2</sub>*  | *CFGKMNOPSUVWXY[\]bh*    |
| *t<sub>3</sub>*  | *DFHKLOPQUX[]`adefgi*    |
| *t<sub>4</sub>*  | *ADIJKPSZ[\`abce*    |
| *t<sub>5</sub>*  | *ABDEFHLPRUX\abce*    |
| *t<sub>6</sub>*  | *BEHJKLNPQX[\^_`cdej*    |
| *t<sub>7</sub>*  | *ADGILORSXYZ[]^fhj*    |
| *t<sub>8</sub>*  | *BEFHLMNOTV[\^afgh*    |
| *t<sub>9</sub>*  | *ABCEGHIJLMOPQRTWXZ[\acdef*   |
| *t<sub>10</sub>* | *CFGINPQRTY_`bg*   |



**2.1**. Каков размер области поиска наборов элементов, если ограничиваться только наборами, состоящими из простых элементов?

**2.2**. Минимальный уровень поддержки равен **5 / 10**. Найдите все часто встречающиеся наборы элементов, состоящие только из элементов высокого уровня в таксономии. Имейте в виду, что если в транзакции появляется простой элемент, предполагается, что все его предки высокого уровня также присутствуют в транзакции. При выполнении данного пункта воспользуйтесь собственной реализацией **Apriori**.

In [37]:
encoder = TransactionEncoder()

dataset = ["BCFGHILNQTUW[_h",
           "CFGKMNOPSUVWXY[]bh",
           "DFHKLOPQUX[]`adefgi",
           "ADIJKPSZ[`abce",
           "ABDEFHLPRUX\\abce",
           "BEHJKLNPQX[^_`cdej",
           "ADGILORSXYZ[]^fhj",
           "BEFHLMNOTV[^afgh",
           "ABCEGHIJLMOPQRTWXZ[\\acdef",
           "CFGINPQRTY_`bg"]

tid = ["t1", "t2", "t3", "t4", "t5", "t6", "t7", "t8", "t9", "t10"]

listds = encoder.fit(dataset).transform(dataset)
dataTask2 = pd.DataFrame(listds, columns = encoder.columns_, index = tid)
print("Размер области = ", dataTask2.shape[1])

taxonomy = {1: ['_','h'], 2: ['b','h'], 3: ['a','b','c','e','f','g','i'], 4: ['d','e','f','g','j']}

def check_item_in_taxonomy(taxonomy, taxon, item):
    if taxon in taxonomy:
        return any(check_item_in_taxonomy(taxonomy, i, item) for i in taxonomy[taxon])
    else:
        return taxon in item

mask = [([check_item_in_taxonomy(taxonomy, i, item) for i in taxonomy]) for item in dataset]
dflast = pd.DataFrame(mask, columns = [1, 2, 3, 4], index = tid)
display(dflast)

def apriori(df: pd.DataFrame, min_support: float) -> Dict[int, List[List[str]]]:
    transaction_matrix = df.values
    transaction_count = transaction_matrix.shape[0]
    support_counts = np.array(np.sum(transaction_matrix, axis=0) / transaction_count)
    item_id_to_name = {i: item for i, item in enumerate(df.columns)}
    item_ids = np.arange(transaction_matrix.shape[1])
    item_set_supports = {1: support_counts[support_counts >= min_support]}
    frequent_item_sets = {1: item_ids[support_counts >= min_support].reshape(-1, 1)}

    k = 2
    while True:
        new_item_sets = []
        leaves = set(frequent_item_sets[k-1].flatten())
        for item_set in frequent_item_sets[k-1]:
            max_item_id = max(item_set)
            new_leaves = [x for x in leaves if x > max_item_id]
            last_tuple = tuple(item_set)
            for new_leaf in new_leaves:
                new_item_sets.append(last_tuple + (new_leaf,))
        
        new_item_sets = np.array(new_item_sets)
        if (new_item_sets.size == 0):
            break
        
        bool_mask = np.array(np.all(transaction_matrix[:, new_item_sets], axis=2))
        support_counts = np.array(np.sum(bool_mask, axis=0) / transaction_count)
        mask = (support_counts >= min_support)
        if mask.any():
            item_set_supports[k] = np.array(support_counts[mask])
            frequent_item_sets[k] = np.array(new_item_sets[mask])
        else:
            break
        
        k += 1
        
    result = {}
    for k in frequent_item_sets:
        for item_set, support in zip(frequent_item_sets[k], item_set_supports[k]):
            result[tuple(item_id_to_name[i] for i in item_set)] = support
    return result

print("\n3, 4:")
display(apriori(dflast[[3, 4]], 0.5))
print("\n1, 2:")
display(apriori(dflast[[1, 2]], 0.5))
print("\nВсего:")
display(apriori(dflast, 0.5))

Размер области =  42


,1,2,3,4
t1,True,True,False,False
t2,True,True,True,False
t3,False,False,True,True
t4,False,True,True,True
t5,False,True,True,True
t6,True,False,True,True
t7,True,True,True,True
t8,True,True,True,True
t9,False,False,True,True
t10,True,True,True,True



3, 4:


{(3,): 0.9, (4,): 0.8, (3, 4): 0.8}


1, 2:


{(1,): 0.6, (2,): 0.7, (1, 2): 0.5}


Всего:


{(1,): 0.6,
 (2,): 0.7,
 (3,): 0.9,
 (4,): 0.8,
 (1, 2): 0.5,
 (1, 3): 0.5,
 (2, 3): 0.6,
 (2, 4): 0.5,
 (3, 4): 0.8,
 (2, 3, 4): 0.5}